In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import string
import itertools
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors
from Levenshtein import distance as lev
from ast import literal_eval
import pickle
from sklearn.model_selection import KFold
import fasttext


In [2]:
noteEvents=pd.read_csv("mimic-iii-clinical-database-1.4/NOTEEVENTS.csv")
diagnosis = pd.read_csv("mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv")

C:\Users\balle\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
categories=set(noteEvents["CATEGORY"].tolist())
categories

{'Case Management ',
 'Consult',
 'Discharge summary',
 'ECG',
 'Echo',
 'General',
 'Nursing',
 'Nursing/other',
 'Nutrition',
 'Pharmacy',
 'Physician ',
 'Radiology',
 'Rehab Services',
 'Respiratory ',
 'Social Work'}

# Preprocessing

In [4]:
noteEvents=noteEvents[["HADM_ID","TEXT","DESCRIPTION","CATEGORY"]]
diagnosis = diagnosis[["HADM_ID","SUBJECT_ID","ICD9_CODE"]]

def exclude_procedure_code(x):
    if str(x).startswith("0"):
        return False
    return True

#Group Diagnosis into 1 list per admission
diagnosis=diagnosis[diagnosis["ICD9_CODE"].apply(lambda x: exclude_procedure_code(x))]

In [5]:
def convert_icd9(x):
    converted_code = []
    for code in x:
        icd9_str = str(code)
        if icd9_str.startswith("E"):
            converted = icd9_str[:4]
        else:
            converted = icd9_str[:3]
        converted_code.append(converted)
    return converted_code

def checkdiagnosis(x):
    for code in x:
        if str(code)[0:3]=="250":
            return True
    return False

def exclude_procedure_code(x):
    included_code = []
    for code in x:
        icd9_str = str(code)
        if icd9_str.startswith("0"):
            pass
        else:
            included_code.append(code)
    return included_code

In [6]:
diagnosisGrouped = diagnosis.groupby(["SUBJECT_ID","HADM_ID"])["ICD9_CODE"].apply(list).reset_index()
diagnosisGrouped.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,2,163353,"[V3001, V053, V290]"
1,3,145834,"[78559, 5849, 4275, 41071, 4280, 6826, 4254, 2..."
2,4,185777,"[1363, 7994, 2763, 7907, 5715, V090, E9317]"
3,5,178980,"[V3000, V053, V290]"
4,6,107064,"[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1..."


In [7]:
diagnosisGrouped["ICD9_CODE"] = diagnosisGrouped["ICD9_CODE"].apply(lambda x: exclude_procedure_code(x))
admissionswithDiabetes = diagnosisGrouped[diagnosisGrouped["ICD9_CODE"].apply(lambda x: checkdiagnosis(x))]
admissionswithDiabetes["ICD9_CODE_primary"] = admissionswithDiabetes["ICD9_CODE"].apply(lambda x: convert_icd9(x))
admissionswithDiabetes.head()

C:\Users\balle\AppData\Local\Temp/ipykernel_8240/642132842.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissionswithDiabetes["ICD9_CODE_primary"] = admissionswithDiabetes["ICD9_CODE"].apply(lambda x: convert_icd9(x))


,SUBJECT_ID,HADM_ID,ICD9_CODE,ICD9_CODE_primary
11,13,143045,"[41401, 4111, 25000, 4019, 2720]","[414, 411, 250, 401, 272]"
15,18,188822,"[25080, 78039, 29633, V5867, E9323, V5869, 478...","[250, 780, 296, V58, E932, V58, 478, 780, 783,..."
17,20,157681,"[41401, 4111, 25000, 2724, 4019]","[414, 411, 250, 272, 401]"
18,21,109451,"[41071, 78551, 5781, 5849, 40391, 4280, 4592, ...","[410, 785, 578, 584, 403, 428, 459, 507, 427, ..."
19,21,111970,"[78552, 40391, 42731, 70709, 5119, 6823, 99859...","[785, 403, 427, 707, 511, 682, 998, 572, 995, ..."


In [8]:
combineddf=noteEvents.merge(admissionswithDiabetes, on="HADM_ID")
print(combineddf.shape)
combineddf.head()

(406203, 7)


,HADM_ID,TEXT,DESCRIPTION,CATEGORY,SUBJECT_ID,ICD9_CODE,ICD9_CODE_primary
0,121936.0,Admission Date: [**2125-2-9**] D...,Report,Discharge summary,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
1,121936.0,PATIENT/TEST INFORMATION:\nIndication: Aortic ...,Report,Echo,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
2,121936.0,PATIENT/TEST INFORMATION:\nIndication: Aortic...,Report,Echo,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
3,121936.0,Sinus rhythm. Frequent atrial premature beats...,Report,ECG,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
4,121936.0,Rhythm is most likely sinus rhythm with freque...,Report,ECG,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."


In [9]:
processeddf=combineddf.copy()
spacestring=" "*len(string.punctuation.replace("'",""))
dstring="d"*len(string.digits)
processeddf["TEXT"]=processeddf["TEXT"].apply(lambda x: x.translate(str.maketrans(string.punctuation.replace("'",""),spacestring)))
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.translate(str.maketrans(string.digits,dstring)))
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.lower())
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.split())

In [10]:
wordslist=processeddf["TEXT"].tolist()
flatlist=list(itertools.chain(*wordslist))
wordset=set(flatlist)
wordDict=Counter(flatlist)
wordDict5orMore= dict(filter(lambda x: x[1] >=5, wordDict.items())) 

wordDictLessThan5 = dict(filter(lambda x: x[1] <5, wordDict.items()))

In [11]:
print(len(wordDict5orMore)) #53229
print(len(wordDictLessThan5)) #109486

53229
109486


In [12]:
###GENERATE Dictionary for mapping mispelled words
# mapped_dict = dict()

# for w in range(len(wordListLessThan5)):
#     lev_dist = []
#     for i in range(len(wordList5orMore)):
#         lev_dist.append(lev(wordListLessThan5[w], wordList5orMore[i]))
#     print(np.argmin(lev_dist))
#     mapped_dict[wordListLessThan5[w]] = wordList5orMore[np.argmin(lev_dist)]

# print(len(mapped_dict))

# with open('mapDictionary.pkl', 'wb') as f:
#     pickle.dump(mapped_dict, f)

with open('mapDictionary.pkl', 'rb') as f:
    mapped_dict = pickle.load(f)

In [13]:
limited_processeddf = processeddf[processeddf['TEXT'].apply(lambda x: len(x)) > 9]
limited_processeddf = limited_processeddf[limited_processeddf['TEXT'].apply(lambda x: len(x)) < 2200]
print(limited_processeddf.shape)
limited_processeddf.head()


(399623, 7)


,HADM_ID,TEXT,DESCRIPTION,CATEGORY,SUBJECT_ID,ICD9_CODE,ICD9_CODE_primary
1,121936.0,"[patient, test, information, indication, aorti...",Report,Echo,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
2,121936.0,"[patient, test, information, indication, aorti...",Report,Echo,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
3,121936.0,"[sinus, rhythm, frequent, atrial, premature, b...",Report,ECG,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
4,121936.0,"[rhythm, is, most, likely, sinus, rhythm, with...",Report,ECG,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
5,121936.0,"[atrial, fibrillation, intraventricular, condu...",Report,ECG,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."


In [14]:
def map_misspelled(x):
    new_l = [mapped_dict.get(item, item) for item in x]
    return new_l

limited_processeddf["TEXT"] = limited_processeddf["TEXT"].apply(lambda x: map_misspelled(x))

In [15]:
limited_processeddf["ICD9_SET"]= limited_processeddf["ICD9_CODE"].apply(lambda x: set(x))
limited_processeddf["ICD9_primary_SET"]= limited_processeddf["ICD9_CODE_primary"].apply(lambda x: set(x))

In [16]:
Binary Relevance
top_10codes=["427","518","428","584","401","276","414","285","272","585"]
def TopCodes(x):
    toplist=[]
    for topcode in top_10codes:
        for code in x:
            flag=0
            if str(code).startswith(topcode):
                flag=1
        toplist.append(flag)
    return toplist

limited_processeddf["Top_ICD9_Codes"]=limited_processeddf["ICD9_primary_SET"].apply(lambda x: TopCodes(x))
binarycodes=pd.DataFrame(limited_processeddf["Top_ICD9_Codes"].tolist(),columns=["427","518","428","584","401","276","414","285","272","585"])
binarycodes.head()


#Filter codes to only top 10 for comparision to Binary relevance
def TopCodesList(x):
    toplist=[]
    for code in x:
        if str(code).startswith(tuple(top_10codes)):
            toplist.append(code)
    return toplist

limited_processeddf["Top_ICD9_Codes"]=limited_processeddf["ICD9_primary_SET"].apply(lambda x: TopCodesList(x))

In [17]:
#Save to CSV
#limited_processeddf.to_csv("Tokenized.csv")
#binarycodes.to_csv("binarycodes.csv")

In [18]:
binarycodes=pd.read_csv("binarycodes.csv")

## Preprocessed Data Statistics

icd9_list=limited_processeddf["ICD9_CODE_primary"].tolist()
icd9_flatlist=list(itertools.chain(*icd9_list))
print("Num. icd9 primary code",len(icd9_flatlist))
icd9_wordset=set(icd9_flatlist)
icd9_wordDict=Counter(icd9_flatlist)
print("Num. unique icd9 primary code",len(icd9_wordDict))
icd9_re_list=limited_processeddf["ICD9_CODE"].tolist()
icd9_re_flatlist=list(itertools.chain(*icd9_re_list))
print("Num. icd9 regular code",len(icd9_re_flatlist))
icd9_re_wordset=set(icd9_re_flatlist)
icd9_re_wordDict=Counter(icd9_re_flatlist)
print("Num. unique icd9 regular code",len(icd9_re_wordDict))

limited_processeddf["ICD9_SET"]= limited_processeddf["ICD9_CODE"].apply(lambda x: set(x))
limited_processeddf["ICD9_primary_SET"]= limited_processeddf["ICD9_CODE_primary"].apply(lambda x: set(x))
limited_processeddf["NumberofICD9Codes"] = limited_processeddf["ICD9_SET"].apply(lambda x: len(x))
limited_processeddf["NumberofICD9CodesPri"] = limited_processeddf["ICD9_CODE_primary"].apply(lambda x: len(set(x)))
CodesperReport = limited_processeddf["NumberofICD9Codes"].tolist()
avgcodesperReport = sum(CodesperReport) / len(CodesperReport)
PriCodesperReportlist = limited_processeddf["NumberofICD9CodesPri"].tolist()
avgPriCodesperReport = sum(PriCodesperReportlist) / len(PriCodesperReportlist)
print("avgCodesperReport",avgcodesperReport)
print("avgPriCodesperReport",avgPriCodesperReport)

limited_processeddf["NumberofTokens"] = limited_processeddf["TEXT"].apply(lambda x: len(x))
NumberofTokensList=limited_processeddf["NumberofTokens"].tolist()
avgTokensperReport = sum(NumberofTokensList) / len(NumberofTokensList)
print("avgTokensperReport",avgTokensperReport)

print("avgDensityperReport",avgcodesperReport/len(icd9_wordDict))
print("avgPriDensityperReport",avgPriCodesperReport/len(icd9_re_wordDict))

limited_processeddf.head(5)

# Build Model

In [19]:
del processeddf,combineddf,admissionswithDiabetes,noteEvents,diagnosis,diagnosisGrouped

In [20]:
import pandas as pd
import numpy as np
from ast import literal_eval
import sklearn as sk
import string
import itertools
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors
import pickle
from sklearn.model_selection import KFold, train_test_split
from sklearn import preprocessing, metrics
from keras.layers import Dense, Lambda, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Embedding, Activation,Input,concatenate,Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model, Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import tensorflow as tf
from matplotlib import pyplot
from sklearn.metrics import multilabel_confusion_matrix, precision_score, recall_score, f1_score


In [21]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# SET ROLLED UP CODES HERE

In [25]:
tokenized=limited_processeddf
mlb = preprocessing.MultiLabelBinarizer()
y = mlb.fit_transform(tokenized["ICD9_CODE_primary"])
y = pd.DataFrame(y)
print(y.shape) #full: (399623, 808)

(399623, 808)


In [26]:
def f1_evaluation_score(y_true, y_pred):
   
    # compute true positives across training examples and labels
    tp = np.sum(np.logical_and(y_true, y_pred))
    
    # compute false negatives (Missed Labels) across training examples and labels
    fn = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))
    
    fp = np.sum(np.logical_and(np.logical_not(y_true), y_pred))

    # compute False Positive across training examples and labels.
    f1 = 2*tp / (2* tp + fp + fn)
    
    return np.mean(f1)

In [27]:
def build_w2v_model(x_train):
    #w2v_model = Word2Vec.load("drive/MyDrive/BD4H Project Folder/word2vec.model")
    w2v_model = Word2Vec(x_train, vector_size=300)
    #w2v_model.save("word2vec_test.model")
    #print(w2v_model)
    return w2v_model

In [28]:
def get_sequence(w2v_model,x_train,x_val,x_test):
    num_unique_tokens = len(w2v_model.wv)
    token = Tokenizer(num_unique_tokens)
    token.fit_on_texts(x_train)
    train_text = token.texts_to_sequences(x_train)
    x_train_seq = pad_sequences(train_text)
    print(x_train_seq.shape) 

    sequences_val = token.texts_to_sequences(x_val)
    x_val_seq = pad_sequences(sequences_val, maxlen=x_train_seq.shape[1]) 
    print(x_val_seq.shape)

    sequences_test = token.texts_to_sequences(x_test)
    x_test_seq = pad_sequences(sequences_test, maxlen=x_train_seq.shape[1]) 
    print(x_test_seq.shape)

    return x_train_seq,x_val_seq,x_test_seq

In [29]:
def build_embedding_vector(w2v_model):

    num_unique_tokens = len(w2v_model.wv)
    token = Tokenizer(num_unique_tokens)
    embeddings_index = {}
    for w in w2v_model.wv.key_to_index:
        idx=w2v_model.wv.key_to_index[w]
        embeddings_index[idx] = w2v_model.wv[idx]

    num_words = num_unique_tokens
    embedding_matrix = np.zeros((num_words, 300))
    for word, i in token.word_index.items():
        if i >= num_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [30]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=5024)])
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)  
        


In [31]:
def create_mlp(dim):
	# define our MLP network
	cat_model = Sequential()
	cat_model.add(Dense(128, input_dim=dim, activation="relu"))
	cat_model.add(Dense(64, activation="relu"))
	return cat_model

In [32]:
def BOT_base_test(x_train_seq,y_train,x_val_seq,y_val,x_test_seq,y_test,num_unique_tokens):
    report_input = Input(shape=(x_train_seq.shape[1],), dtype='int32')

    report_encoder = Embedding(num_unique_tokens, 300, input_length=x_train_seq.shape[1], trainable=True)(report_input)
    report_encoder_mean = Lambda(lambda x: tf.reduce_mean(x, axis=1, keepdims=True))(report_encoder)
    bigram_branch = Dense(y_train.shape[1])(report_encoder_mean)
    bigram_branch = GlobalMaxPooling1D()(report_encoder_mean)

    merged = Dense(y_train.shape[1])(bigram_branch)

    output = Activation('sigmoid')(merged)

    model = Model(inputs=[report_input], outputs=[output])
    
    return model

In [33]:
def concatenate_3dense(mlp,cnn,cat_train,cat_val,cat_test,x_train_seq,y_train,x_val_seq,y_val,x_test_seq,y_test):
    combinedInput = concatenate([mlp.output, cnn.output])
    
    #Add/Remove Dense Layers Here:
    
    combinedInput = Dense(64, activation="relu")(combinedInput)
    #combinedInput = Dense(64, activation="relu")(combinedInput)
    combinedInput = Dense(64, activation="relu")(combinedInput)
    
    combinedOutput = Dense(y_train.shape[1], activation="sigmoid")(combinedInput)
    model = Model(inputs=[mlp.input, cnn.input], outputs=combinedOutput)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss=tf.keras.losses.BinaryCrossentropy())
    
    callbacks = [EarlyStopping(min_delta=0.0001,patience=2)]
    model.fit([cat_train, x_train_seq], y_train, validation_data=([cat_val, x_val_seq], y_val),callbacks=callbacks,batch_size=32, epochs=2)

    yhat_cnn = model.predict([cat_test, x_test_seq])
    yhat_cnn[yhat_cnn>0.5] = 1
    yhat_cnn[yhat_cnn<=0.5] = 0
    yhat_cnn = pd.DataFrame(yhat_cnn,dtype=int)

    precision = precision_score(y_test, yhat_cnn,average="micro")
    recall = recall_score(y_test, yhat_cnn,average="micro")
    f1 = f1_score(y_test, yhat_cnn, average="micro")
    return  precision, recall, f1

In [34]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [35]:
ohe = preprocessing.OneHotEncoder()
cats = ohe.fit_transform(tokenized[["CATEGORY"]]).toarray()
print(cats.shape)
cats = pd.DataFrame(cats)

(399623, 15)


In [ ]:
tokenized=tokenized[["TEXT"]]
def runBOTbase():    

    kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
    precision_list_re = []
    recall_list_re = []
    f1_list_re = []

    for train_index, test_index in kf.split(tokenized):
        #print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = tokenized["TEXT"].iloc[train_index], tokenized["TEXT"].iloc[test_index]
        y_train_regular,y_test_regular = y.iloc[train_index],y.iloc[test_index]
        cat_train,cat_test =cats.iloc[train_index], cats.iloc[test_index]
        x_val = x_train.sample(frac=0.1,random_state=42)
        y_val_regular = y_train_regular.sample(frac=0.1,random_state=42)
        cat_val = cat_train.sample(frac=0.1,random_state=42)

        w2v_model = build_w2v_model(x_train)
        num_unique_tokens = len(w2v_model.wv)
        x_train_seq,x_val_seq,x_test_seq = get_sequence(w2v_model,x_train,x_val,x_test)
        embedding_matrix = build_embedding_vector(w2v_model)
        mlp = create_mlp(cat_train.shape[1])

        BOT_base = BOT_base_test(x_train_seq,y_train_regular,x_val_seq,y_val_regular,x_test_seq,y_test_regular,num_unique_tokens)
        
        precision, recall, f1 = concatenate_3dense(mlp,BOT_base,cat_train,cat_val,cat_test,x_train_seq,y_train_regular,x_val_seq,y_val_regular,x_test_seq,y_test_regular)

        print(precision, recall, f1)
        precision_list_re.append(precision)
        recall_list_re.append(recall)
        f1_list_re.append(f1)
        tf.keras.backend.clear_session()
    
    print(np.mean(precision_list_re))
    print(np.mean(recall_list_re))
    print(np.mean(f1_list_re))


        
runBOTbase()



print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(gpus)

In [ ]:
#Reset Tokenized if running samples
tokenized=limited_processeddf

In [ ]:
tf.keras.backend.clear_session()